In [ ]:
import sys
sys.path.append("D:\\ASGaze")

import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
from gaze_ray_estimator.estimator import get_cone, load_files
from mapping.shape_constrained import cal_offsets
from mapping.mapping_principle import Line3

### Step 1: Ambiguity Removal

In [ ]:
def calAngle(a,b):
    unit_a = a / np.linalg.norm(a)
    unit_b = b / np.linalg.norm(b)
    dot_product = np.dot(unit_a, unit_b)
    rad = np.arccos(dot_product)    
    return rad

def ambiguity_removal(gaze_direction,start,end,window):
    angle_0_list = []
    angle_1_list = []

    for i in range(start+window,end,window):
        angle_0 = calAngle(gaze_direction[i,0],gaze_direction[start,0])
        angle_1 = calAngle(gaze_direction[i,1],gaze_direction[start,1])
        angle_0_list.append(angle_0)
        angle_1_list.append(angle_1)
        
    x = np.arange(start+window,end,window)
    l1=plt.plot(x,angle_0_list,'r--',label='angle_0')
    l2=plt.plot(x,angle_1_list,'g--',label='angle_1')
    plt.plot(x,angle_0_list,'ro-',x,angle_1_list,'g+-')
    plt.xlabel('Distance change from start point')
    plt.ylabel('Cumulative rotation differences')
    plt.legend()
    plt.show()
    
    angle_0_np = np.array(angle_0_list)
    angle_1_np = np.array(angle_1_list)
    if(np.sum(angle_0_np) > np.sum(angle_1_np)):
        true_seq = 1
    else:
        true_seq = 0
        
    return true_seq

In [ ]:
def ambiguity_removal_main(p_root,test_name,model_id,iris_radius):

    _,frame_files,ellipse_files_l,ellipse_files_r,crop_info,_,camera_matrix_inv = load_files(p_root,test_name,model_id)
    gaze_l_list = []
    gaze_r_list = []

    for i in range(0,len(frame_files)):
        _,_,gaze_l,gaze_r,_,_ = get_cone(ellipse_files_l[i],ellipse_files_r[i],crop_info[i],camera_matrix_inv,iris_radius)
        gaze_l_list.append(gaze_l)
        gaze_r_list.append(gaze_r)

    gaze_l_np = np.array(gaze_l_list)
    gaze_r_np = np.array(gaze_r_list)

    start = 0
    end = len(frame_files)
    window = 20
    true_seq = ambiguity_removal(gaze_l_np,start,end,window)
    print("True sequence is:",true_seq)
    
    return true_seq

### Step 2: Calculate offsets between CCS and TCS

In [ ]:
def cal_offsets_main(p_root,test_name_list,model_id,iris_radius,true_seq,thresholds):
    PoG_mean_list = []
    # T look-up table
    # Reference from https://www.scribbr.com/statistics/students-t-table/
    t = [None,12.706,4.303,3.182,2.776,2.571,2.447,2.365,2.306,2.262,2.228,2.201,2.179,2.160,2.145,2.131,2.120,2.110,2.101,2.093,2.086,2.080,2.074,2.069,2.064,2.060,2.056,2.052,2.048,2.045,2.042]

    for idx,test_name in enumerate(test_name_list):
        _,frame_files,ellipse_files_l,ellipse_files_r,crop_info,_,camera_matrix_inv = load_files(p_root,test_name,model_id)
        PoG_list = []
        for i in range(0,len(frame_files)):
            _,_,gaze_l,gaze_r,gaze_origin_l,gaze_origin_r = get_cone(ellipse_files_l[i],ellipse_files_r[i],crop_info[i],camera_matrix_inv,iris_radius)
            gaze_l = gaze_l[true_seq] 
            gaze_r = gaze_r[true_seq]

            gaze_origin_l = gaze_origin_l[true_seq]
            gaze_origin_r = gaze_origin_r[true_seq]

            gaze_line_l = Line3(p=gaze_origin_l[:3], normal=gaze_l)
            PoG_l = gaze_line_l.get_pos() # CCS
            gaze_line_r = Line3(p=gaze_origin_r[:3], normal=gaze_r)
            PoG_r = gaze_line_r.get_pos() # CCS

            PoG = 0.5*(PoG_l+PoG_r)
            PoG_list.append(PoG)
            
            if(i > 2):
                df = i-1
                
                standard_x = np.std(np.array(PoG_list)[:i,0])
                temp_x = (standard_x/np.sqrt(i))*t[df]
                
                standard_y = np.std(np.array(PoG_list)[:i,1])
                temp_y = (standard_y/np.sqrt(i))*t[df]
                
                if((temp_x <= thresholds[idx]) and (temp_y) <= thresholds[idx]):
                    PoG_mean_list.append(np.mean(np.array(PoG_list)[:,:2],axis=0))
                    break
                    
    PoG_np = np.array(PoG_mean_list)
    index,offsets = cal_offsets(PoG_np)
    print("offsets",offsets)
    
    return offsets